## 1. get top 100 hashtags

In [1]:
import glob
import csv
import os
import re
import datetime
import calendar
import configparser 

import time
import datetime
import pymongo
from pymongo import MongoClient

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# set folder name in config.ini
config = configparser.ConfigParser()
config.read('config.ini')
folder_name = config['DEFAULT']['Folder-Name']
ip = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']

In [2]:
# read merged csv file
folder = "output/merged_csv/"
input_file = folder + folder_name + "_merged_data.csv"

dic_hashtag = {}
with open(input_file,'r') as csv_file:
    file_reader = csv.reader(csv_file,delimiter=',')
    header = next(file_reader,None)
    for row in file_reader:
        hashtag = row[0]
        num_tweet = row[4]
        if hashtag in dic_hashtag :
            dic_hashtag[hashtag] = int(dic_hashtag[hashtag]) + int(num_tweet)
        else:
            dic_hashtag[hashtag] = int(num_tweet)

print("counting hashtag number for all collections is finished")

counting hashtag number for all collections is finished


In [3]:
# filter out the irrelevant hashtag from the dictionary
filter_list = ["sydney","australia","melbourne","brisbane","perth","adelaide","romance","hottest100","jimin"]
for i in filter_list:
    try:
        del dic_hashtag[i]  
    except:
        pass

In [4]:
# get top 100 hashtags
top_100_htag = dict(sorted(dic_hashtag.items(), key=lambda x: x[1], reverse=True)[:100])
print(top_100_htag)

{'auspol': 284, 'tearitupbts': 212, 'grammys': 203, 'mafsau': 77, 'layzhanggrammys': 63, 'mafs': 59, 'qanda': 46, 'medivacbill': 45, 'manus': 44, 'mkr': 34, 'btsarmy': 33, 'iheartawards': 32, 'savehakeem': 26, 'nauru': 25, 'breaking': 24, 'infoonline19': 24, 'bts': 22, 'medevacbill': 21, 'btsxgrammys': 19, 'qt': 18, 'bestfanarmy': 18, 'backthebill': 17, 'climatechange': 17, 'nowplaying': 17, 'hakeemalaraibi': 16, 'quote': 16, '9news': 15, 'asmsg': 15, 'abc730': 15, 'valentinesday': 14, 'npc': 14, 'paladinaffair': 13, '4yearsforculttemper': 13, 'iartg': 13, 'thedrum': 12, 'womeninscience': 12, 'kindle': 11, 'asxstockbot': 11, 'dutton': 11, 'blackpinkongma': 11, '5sosfam': 11, 'ian1': 11, 'imacelebrityau': 11, 'grammy2019': 11, 'iranian': 10, 'canberra': 10, 'photography': 10, 'bankingrc': 10, 'spub': 10, 'pmlive': 10, 'nintendodirect': 9, 'news': 9, 'morrison': 9, 'lnp': 9, 'btsxgrammys2019': 9, 'ausbiz': 9, 'twitch': 8, 'googleads': 8, 'nrlallstars': 8, 'bitcoin': 8, 'tweet4taiji': 8, 

## 2. get top 100 tweets with the most reply for each top 100 hashtags per week

In [5]:
# connect to mongodb
client = MongoClient(ip, int(port))
db_twitter = client["Twitter"]
collections_twitter = db_twitter.collection_names()

In [6]:
current_timestamp = int(time.time() * 1000)
current_year = int(datetime.datetime.now().year)
print("current year : " + str(current_year))

current_week = int((current_timestamp - 1546214400000)/1000/604800)+1
print("current week : " + str(current_week))

current year : 2019
current week : 14


In [7]:
dic_collection = {}
for i in collections_twitter:
    if i.startswith("20"):
        year = i[0:4]
        week = re.search('_(.+?)_', i).group(1)[1:]
        if int(year) < current_year:
            dic_collection[i] = "{:}".format(db_twitter[i].find({}).count())
        else:
            try:
                if int(week) < current_week:
                    dic_collection[i] = "{:}".format(db_twitter[i].find({}).count())
            except: pass

for key in sorted(dic_collection):
    print("%s: %s" % (key, dic_collection[key]))

2018_W51_Twitter_Australia: 20471
2018_W51_Twitter_Other: 8978243
2018_W52_Twitter_Australia: 38065
2018_W52_Twitter_Other: 16940708
2019_W01_Twitter_Australia: 40880
2019_W01_Twitter_Other: 16994959
2019_W02_Twitter_Australia: 37645
2019_W02_Twitter_Other: 14474627
2019_W03_Twitter_Australia: 52348
2019_W03_Twitter_Other: 20416992
2019_W04_Twitter_Australia: 59625
2019_W04_Twitter_Other: 23417927
2019_W05_Twitter_Australia: 61617
2019_W05_Twitter_Other: 23917345
2019_W06_Twitter_Australia: 61435
2019_W06_Twitter_Other: 24176122
2019_W07_Twitter_Australia: 28017
2019_W07_Twitter_Other: 10484648
2019_W08_Twitter_Australia: 60035
2019_W08_Twitter_Other: 23446630
2019_W09_Twitter_Australia: 58316
2019_W09_Twitter_Other: 22293795
2019_W10_Twitter_Australia: 40767
2019_W10_Twitter_Other: 15941350
2019_W11_Twitter_Australia: 68593
2019_W11_Twitter_Other: 25345811
2019_W12_Twitter_Australia: 50447
2019_W12_Twitter_Other: 18090490
2019_W13_Twitter_Australia: 62926
2019_W13_Twitter_Other: 24379

In [8]:
# delete existed collection from the list dic_collection
def delete_collection(folder,dic_collection):
    for input_file in glob.glob(os.path.join(folder,'*.csv')):
        collection_name = re.search('{(.+?)}', input_file).group(1)
        print("Existed collection: " + collection_name)
        del dic_collection[collection_name]
    return dic_collection

In [9]:
# create foler if not exist
def create_folder():
    folder = "output/hashtag_100_tweets/"
    if not os.path.exists(folder):
        os.makedirs(folder)
    return folder

In [10]:
# write csv file
def write_csv(collection,data_format,folder):
    
    file_name = folder + "{" + collection + "}_100_tweets.csv"
    with open(file_name, 'w') as f:
        # header
        f.write('hashtag,tweet,retweet_count,reply_count\n')
            
        for content in data_format:
            hashtag = content['hashtag']
            tweet = content['text']
            retweet_count = content['retweet_count']
            reply_count = content['reply_count']

            # prevent string from splitting on comma when writing a csv file
            f.write('"{}","{}","{}","{}"'.format(hashtag, tweet, retweet_count, reply_count))
            f.write("\n")
    return file_name       

In [11]:
#create folder if not exist
folder = create_folder()
    
# delete existed collection from the list dic_collection
dic_collection = delete_collection(folder,dic_collection)

for collection in sorted(dic_collection):
    
    data_format = []
    
    print("-----------------------" + "\n")
    print("processing collection " + collection + " ....")
    for h in top_100_htag:
        for tweet in db_twitter[collection].find({"$text": {"$search": "\"#" + h + "\"", "$caseSensitive": False}}).sort([("retweet_count", pymongo.DESCENDING),("reply_count", pymongo.DESCENDING)]).limit(100):
            dic_top_tweets = {}
            
            dic_top_tweets["hashtag"] = h         
            dic_top_tweets["text"] = tweet["text"].replace("\n"," ").replace(","," ")        
            
            if "retweeted_status" in tweet :
                dic_top_tweets["retweet_count"] = tweet["retweeted_status"]["retweet_count"]
                dic_top_tweets["reply_count"] = tweet["retweeted_status"]["reply_count"]
            else:
                dic_top_tweets["retweet_count"] = tweet["retweet_count"]
                dic_top_tweets["reply_count"] = tweet["reply_count"]
                        
            data_format.append(dic_top_tweets)
    print("list of top 100 tweets for collection " + collection + " is finished.")
    
    file_name = write_csv(collection,data_format,folder)
    print("File " + file_name + " is ready.")
    print("-----------------------" + "\n")

Existed collection: 2019_W05_Twitter_Australia
Existed collection: 2019_W08_Twitter_Australia
Existed collection: 2019_W07_Twitter_Australia
Existed collection: 2018_W51_Twitter_Australia
Existed collection: 2018_W52_Twitter_Australia
Existed collection: 2019_W04_Twitter_Australia
Existed collection: 2019_W09_Twitter_Australia
Existed collection: 2019_W06_Twitter_Australia
Existed collection: 2019_W11_Twitter_Australia
Existed collection: 2019_W02_Twitter_Australia
Existed collection: 2019_W13_Twitter_Australia
Existed collection: 2019_W01_Twitter_Australia
Existed collection: 2019_W10_Twitter_Australia
Existed collection: 2019_W12_Twitter_Australia
Existed collection: 2019_W03_Twitter_Australia
-----------------------

processing collection 2018_W51_Twitter_Other ....
list of top 100 tweets for collection 2018_W51_Twitter_Other is finished.
File output/hashtag_100_tweets/{2018_W51_Twitter_Other}_100_tweets.csv is ready.
-----------------------

-----------------------

processing coll

KeyboardInterrupt: 